# Display images from telescopes

In [ ]:
import ctapipe
print(ctapipe.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ctapipe.instrument import SubarrayDescription
from ctapipe.visualization import CameraDisplay
from ctapipe.io import TableLoader
from pathlib import Path

In [ ]:
dl2_proton_filename = "../data/proton_with_images_00.dl2.h5"
dl2_gamma_filename = "../data/gamma-diffuse_with_images_00.dl2.h5"

In [ ]:
subarray = SubarrayDescription.from_hdf(dl2_proton_filename)

In [ ]:
def read_table(filename, tel_ids=None, stop=None):
    """
    Read data from a file and return it as a pandas DataFrame.

    Parameters
    ----------
    filename : str
        The path to the input file.
    tel_ids : list or None, optional
        A list of telescope IDs to include in the data. If None, all telescopes are included. Default is None.
    stop : int or None, optional
        The number of events to read from the file. If None, all events are read. Default is None.
    Returns
    -------
    pandas.DataFrame
        The data read from the file as a pandas DataFrame.
    """
    loader = TableLoader(
        input_url=filename,
        load_dl1_parameters=True,
        load_dl2=False,
        load_instrument=False,
        load_simulated=False,
        load_true_parameters=False,
        load_dl1_images=True,
    )

    data = loader.read_telescope_events(telescopes=tel_ids, stop=stop)
    return data
    

In [ ]:
def frame_strip(display):
    """
    Remove all frame elements from a CameraDisplay object.

    Parameters
    ----------
    display : ctapipe.visualization.CameraDisplay
        The CameraDisplay object to modify.
    """
    display.axes.set_frame_on(False)  # Remove the frame
    display.axes.set_title('')  # Remove the title
    display.axes.set_xticks([])  # Remove x-axis ticks
    display.axes.set_yticks([])  # Remove y-axis ticks
    display.axes.set_xlabel('')  # Remove x-axis label
    display.axes.set_ylabel('')  # Remove y-axis label
    display.add_frame_name((1, 1, 1, 0))  # Remove "CameraFrame" by setting alpha to 0
    display.cmap.set_bad(alpha=0)  # Make the background transparent
    for text in display.axes.texts:
        if "Frame" in text.get_text():  # Match the label text
            text.set_visible(False)  # Hide it

In [ ]:
protons = read_table(dl2_proton_filename, stop=1000)
gammas = read_table(dl2_gamma_filename, stop=1000)


In [ ]:
# select events above a certain intensity threshold
min_intensity = 100
protons = protons[protons['hillas_intensity'] > min_intensity]
gammas = gammas[gammas['hillas_intensity'] > min_intensity]

print(f"Number of protons: {len(protons)}")
print(f"Number of gammas: {len(gammas)}")

# Random Events

In [ ]:
def plot_images_board(events):
    fig, axes = plt.subplots(6, 10, figsize=(16, 9))

    for i in range(60):
        ax = axes.flatten()[i]
        image = events[i]['image']
        tel_id = events[i]['tel_id']
        camera = subarray.tel[tel_id].camera
        geometry = camera.geometry
        display = CameraDisplay(geometry, ax=ax)
        display.image = image
        frame_strip(display)

    plt.subplots_adjust(wspace=0, hspace=0)  # Remove space between subplots
    plt.tight_layout()

In [ ]:
plot_images_board(gammas)
plt.title('Gamma events')
plt.show()

In [ ]:
plot_images_board(protons)
plt.title('Proton events')
plt.show()
